# Human Parsing with SegFormer B5 Model
This notebook demonstrates how to use the `segformer-b5-finetuned-human-parsing` model from Hugging Face to perform human parsing, which segments human images into various categories like hair, face, clothes, etc.

In [16]:
pip install transformers torch

  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/cf/90/2596ac2ab49c4df6ff1fceaf7f5afb18401ba2f326348ce1a6261a65e7ed/transformers-4.40.1-py3-none-any.whl.metadata
  Using cached transformers-4.40.1-py3-none-any.whl.metadata (137 kB)
  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/37/04/a5cd83baccbf2d4329990ec06b8abf3a644e1559a7b1f764f42d2cb77d51/torch-2.3.0-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for filelock from https://files.pythonhosted.org/packages/6e/b5/15b3b36f298bcbc0be82a371ac744f4f5a10309ade0b8bbde286598dd612/filelock-3.13.4-py3-none-any.whl.metadata
  Using cached filelock-3.13.4-py3-none-any.whl.metadata (2.8 kB)
  Obtaining dependency information for huggingface-hub<1.0,>=0.19.3 from https://files.pythonhosted.org/packages/05/c0/779afbad8e75565c09ffa24a88b5dd7e293c92b74eb09df6435fc58ac986/huggingface_hub-0.22.2-py3-none-any.whl.metadata
  Using cached hugg


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


## Import Libraries

In [1]:
from transformers import SegformerForSemanticSegmentation, SegformerFeatureExtractor
import torch
from PIL import Image
import requests
from io import BytesIO
import os

c:\Users\lance\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
import torch
print(torch.cuda.is_available())
print(torch.version.cuda)

True
12.1


## Load Model and Feature Extractor

In [18]:
model_directory = '../Models/segformer-b5-finetuned-human-parsing'
model_id = 'matei-dorian/segformer-b5-finetuned-human-parsing'
def load_model_and_extractor(model_dir, model_id):
    # Check if the directory exists and is not empty
    required_files = ['config.json', 'pytorch_model.bin']
    files_present = os.listdir(model_dir) if os.path.exists(model_dir) else []

    # Check if all required files are in the directory
    if all(file in files_present for file in required_files):
        print("Loading model and feature extractor from local directory.")
        model = SegformerForSemanticSegmentation.from_pretrained(model_dir)
        feature_extractor = SegformerFeatureExtractor.from_pretrained(model_dir)
    else:
        print("Local directory is empty or missing files. Downloading model and feature extractor.")
        model = SegformerForSemanticSegmentation.from_pretrained(model_id)
        feature_extractor = SegformerFeatureExtractor.from_pretrained(model_id)
        
        # Optionally save the model and feature extractor locally for future use
        model.save_pretrained(model_dir)
        feature_extractor.save_pretrained(model_dir)

    return model, feature_extractor
model, feature_extractor = load_model_and_extractor(model_directory, model_id)

Local directory is empty or missing files. Downloading model and feature extractor.


c:\Users\lance\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\models\segformer\feature_extraction_segformer.py:28: FutureWarning: The class SegformerFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use SegformerImageProcessor instead.
  warnings.warn(


## Prepare the Image

In [3]:
img_path = "test_image.jpg"
image = Image.open(img_path)
inputs = feature_extractor(images=image, return_tensors='pt')

## Perform Inference

In [4]:
with torch.no_grad():
    outputs = model(**inputs)
segmentation_mask = outputs.logits.argmax(dim=1).squeeze().numpy()

## Visualize the Result

In [12]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

def decode_segmentation_mask(mask, labels):
    # Define a color for each label (in RGB)
    label_colors = np.array([
        [0, 0, 0],       # Background - Black
        [255, 192, 203], # Skin - Pink
        [0, 0, 255],     # Hair - Blue
        [255, 0, 0],     # Upper-body clothing - Red
        [0, 255, 0],     # Lower-body clothing - Green
        [0, 255, 255],   # Shoes - Cyan
        # Add additional colors for any other categories
        [255, 255, 0],   # Additional Category 1 - Yellow
        [255, 165, 0],   # Additional Category 2 - Orange
        [128, 0, 128],   # Additional Category 3 - Purple
        [255, 20, 147],  # Additional Category 4 - Deep Pink
        [75, 0, 130],    # Additional Category 5 - Indigo
        [0, 128, 128],    # Additional Category 6 - Teal
        [255, 255, 0],   # Additional Category 1 - Yellow
        [255, 165, 0],   # Additional Category 2 - Orange
        [128, 0, 128],   # Additional Category 3 - Purple
        [255, 20, 147],  # Additional Category 4 - Deep Pink
        [75, 0, 130],    # Additional Category 5 - Indigo
        [0, 128, 128]    # Additional Category 6 - Teal
    ])
    # Ensure label_colors covers all the labels present in the mask
    color_mask = label_colors[mask]
    return Image.fromarray(color_mask.astype(np.uint8))
def apply_mask_on_image(original_image, mask, alpha=0.5):
    # Convert PIL image to array
    image_array = np.array(original_image)
    # Resize mask to match image size
    mask_resized = mask.resize(original_image.size, resample=Image.BILINEAR)
    mask_array = np.array(mask_resized)
    # Blend original image and color mask
    blended_image = (1 - alpha) * image_array + alpha * mask_array
    blended_image = blended_image.astype(np.uint8)
    return Image.fromarray(blended_image)

# Example usage
original_image = Image.open("test_image.jpg")  # Load your original image
labels = ["Background", "Hat", "Hair", "Sunglasses", " Upper-clothes", "Skirt", "Pants", "Dress", "Belt", "Left-shoe", "Right-shoe", "Face", "Left-leg", "Right-leg", "Left-arm", "Right-arm", "Bag", "Scarf"]
decoded_mask = decode_segmentation_mask(segmentation_mask, labels)
blended_image = apply_mask_on_image(original_image, decoded_mask, alpha=0.5)

# Display the result
blended_image.show()


ModuleNotFoundError: No module named 'PIL'